# Quantiles
How do ENSO quantiles change over time?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.crs as ccrs
import numpy as np
import scipy.stats
import seaborn as sns
import xarray as xr
import tqdm
import pathlib
import cmocean
import os
import cartopy.util
import copy

# Import custom modules
import src.utils
from src.XRO import XRO, xcorr

## set plotting specs
sns.set(rc={"axes.facecolor": "white", "axes.grid": False})

## bump up DPI
mpl.rcParams["figure.dpi"] = 100

## get filepaths
DATA_FP = pathlib.Path(os.environ["DATA_FP"])
SAVE_FP = pathlib.Path(os.environ["SAVE_FP"])

## Load data

In [ ]:
## Load T,h (total)
Th_total = xr.open_dataset(DATA_FP / "cesm" / "Th.nc")

## get windowed data
Th_total = src.utils.get_windowed(Th_total, stride=60)

## compute quantiles
quant = Th_total.groupby("time.season").quantile(
    q=[0.05, 0.5, 0.95], dim=["time", "member"]
)
quant = quant.rename({"quantile": "q"})

## compute difference
delta_q = quant - quant.isel(year=0)

In [ ]:
VARNAME = "T_3"
SEASON = "SON"

fig, ax = plt.subplots(figsize=(4, 3))

## plot data
ax.plot(delta_q.year, delta_q[VARNAME].sel(season=SEASON, q=0.05))
ax.plot(delta_q.year, delta_q[VARNAME].sel(season=SEASON, q=0.5))
ax.plot(delta_q.year, delta_q[VARNAME].sel(season=SEASON, q=0.95))

## format
ax.set_xlim([1850, 2000])
ax.set_ylim([-0.2, 1])